In [3]:
import pandas as pd
import numpy as np
import glob
import csv
import os
import re

pd.set_option('display.max_columns',500)

outDir="./res_tmp_full"
alignmentDir=outDir+"/alt_alignments"

blatCols=["matches",
        "misMatches",
        "repMatches",
        "nCount",
        "qNumInsert",
        "qBaseInsert",
        "tNumInsert",
        "tBaseInsert",
        "strand",
        "qName",
        "qSize",
        "qStart",
        "qEnd",
        "tName",
        "tSize",
        "tStart",
        "tEnd",
        "blockCount",
        "blockSizes",
        "qStarts",
        "tStarts"]
intersectCols=["chrom",
                "source",
                "type",
                "start",
                "end",
                "score",
                "strand",
                "phase",
                "attributes",
                "chromK",
                "sourceK",
                "typeK",
                "startK",
                "endK",
                "scoreK",
                "strandK",
                "phaseK",
                "attributesK",
                "distance"]
samCols=['QNAME',
         'FLAG',
         'RNAME',
         'POS',
         'MAPQ',
         'CIGAR',
         'RNEXT',
         'PNEXT',
         'TLEN',
         'SEQ',
         'QUAL']

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]

In [4]:
# functions for the results parsing

def extractFlagBits(data):
    data["paired"]=data["FLAG"]               &1 #template having multiple segments in sequencing
    data["aligned2Mates"]=data["FLAG"]        &2 #each segment properly aligned according to the aligner
    data["unmappedCurr"]=data["FLAG"]         &4 #segment unmapped
    data["unmappedMate"]=data["FLAG"]         &8 #next segment in the template unmapped
    data["reversedCurr"]=data["FLAG"]         &16 #SEQ being reverse complemented
    data["reversedMate"]=data["FLAG"]         &32 #SEQ of the next segment in the template being reverse complemented
    data["firstRead"]=data["FLAG"]            &64 #the first segment in the template
    data["lastRead"]=data["FLAG"]             &128 #the last segment in the template
    data["secondaryAlignment"]=data["FLAG"]   &256 #secondary alignment
    data["noPassFilter"]=data["FLAG"]         &512 #not passing filters, such as platform/vendor quality controls
    data["PCRdup"]=data["FLAG"]               &1024 #PCR or optical duplicate
    data["suppAl"]=data["FLAG"]               &2048 #supplementary alignment

def se(row):
    cigar=row["CIGAR"]
    chars=re.findall(r"[\D']+", cigar)
    ints=[int(x) for x in re.findall(r"[\d']+",cigar)]
    readLen=0
    pre=0
    post=0
    n=0
    m_pre_tem=0
    m_pre_ref=0
    m_post_tem=0
    m_post_ref=0
    indexN=0
    di=0
    blockCount=1
    blockSizes=[0]
    tStarts=[0]
    if "N" in chars:
        indexN=chars.index("N")
        blockCount=len(cigar.split("N"))
    for i in range(len(chars)):
        if i==0 and chars[i] in "SH":
            pre=ints[i]
            readLen=readLen+ints[i]
            tStarts[0]=tStarts[0]+ints[i]
        if i==len(chars)-1 and chars[i] in "SH":
            post=ints[i]
            readLen=readLen+ints[i]
        if chars[i]=="N":
            n=n+ints[i]
            tStarts.append(tStarts[-1]+blockSizes[-1]+n)
            blockSizes.append(0)
        if i<indexN and chars[i]=="M":
            m_pre_tem=m_pre_tem+ints[i]
            m_pre_ref=m_pre_ref+ints[i]
            readLen=readLen+ints[i]
            blockSizes[-1]=blockSizes[-1]+ints[i]
        if i>=indexN and chars[i]=="M":
            m_post_tem=m_post_tem+ints[i]
            m_post_ref=m_post_ref+ints[i]
            readLen=readLen+ints[i]
            blockSizes[-1]=blockSizes[-1]+ints[i]
        if i<indexN and chars[i]=="D":
            m_pre_ref=m_pre_ref+ints[i]
            di=di+ints[i]
        if i>=indexN and chars[i]=="D":
            m_post_ref=m_post_ref+ints[i]
            di=di+ints[i]
        if i<indexN and chars[i]=="I":
            readLen=readLen+ints[i]
            m_pre_tem=m_pre_tem+ints[i]
            di=di+ints[i]
        if i>=indexN and chars[i]=="I":
            readLen=readLen+ints[i]
            m_post_tem=m_post_tem+ints[i]
            di=di+ints[i]
    return pd.Series([pre,post,m_pre_ref,m_pre_tem,m_post_ref,m_post_tem,n,readLen,di,blockCount,",".join([str(x) for x in blockSizes]),",".join([str(x) for x in tStarts])])

def parseCIGAR(data):
    data["CIGAR"].replace("*",np.nan,inplace=True)
    data.dropna(axis=0,inplace=True)
    data.reset_index(drop=True,inplace=True)

#     data["READ_LEN"]=data.SEQ.str.len()
    data["CIGAR_POST"]=data.CIGAR.str.extract("[M]([0-9]+)[A-Z]$",expand=False).replace(np.nan,0).astype(int)
    data["END"]=data.READ_LEN-data.CIGAR_POST
    data["CIGAR_PRE"]=data.CIGAR.str.extract("^([0-9]+)[SH]",expand=False).replace(np.nan,0).astype(int)

    data16=data[data["reversedCurr"]==16].reset_index(drop=True)
    data0=data[data["reversedCurr"]==0].reset_index(drop=True)
    data16["Template_start"]=data16.READ_LEN-data16.END
    data16["Template_end"]=data16.READ_LEN-data16.CIGAR_PRE
    data0["Template_start"]=data0.CIGAR_PRE
    data0["Template_end"]=data0.END

    data16["Reference_start"]=data16.READ_LEN-data16.END+data16.POS-data16.Template_start
    data16["Reference_end"]=data16.READ_LEN-data16.CIGAR_PRE-1+data16.POS-data16.Template_start+data16.N
    data0["Reference_start"]=data0.POS
    data0["Reference_end"]=data0.END+data0.POS-data0.CIGAR_PRE+data0.N 
    
    data=pd.concat([data16,data0]).reset_index(drop=True)
    data.drop(["CIGAR_POST","CIGAR_PRE"],axis=1,inplace=True)
    return data

def tStarts(row):
    re=row["Reference_start"]
    tStarts=[]
    qStarts=row.qStarts.split(",")
    qs2=[int(x)-int(qStarts[0]) for x in qStarts]
    blockSizes=row.blockSizes.split(",")
    for i in range(row["blockCount"]):
        tStarts.append(re+int(qStarts[i]))
    return pd.Series([",".join([str(x) for x in tStarts])])


In [5]:
# get all alignments into a single dataframe tagged by chromosome from SAM files (GMAP)
dfAllSam=pd.DataFrame([])
for sam in os.listdir(alignmentDir):
    if sam[-4:]==".sam":
        chrom=sam.split("_")[0]
        fp=alignmentDir+"/"+sam
        df=pd.read_csv(fp,sep="\t",comment='@',usecols=[0,1,2,3,4,5,6,7,8,9,10],names=samCols)
        df["MD"]=pd.read_csv(fp,usecols=[0],comment="@",names=["full"])["full"].str.split("\tMD:Z:",expand=True)[1].str.split("\t",expand=True)[0]
        extractFlagBits(df)
        dfAllSam=pd.concat([dfAllSam,df],axis=0).reset_index(drop=True)
        
dfAllSam["PRE"]=np.nan
dfAllSam["POST"]=np.nan
dfAllSam["MPRER"]=np.nan
dfAllSam["MPRET"]=np.nan
dfAllSam["MPOSTR"]=np.nan
dfAllSam["MPOSTT"]=np.nan
dfAllSam["N"]=np.nan
dfAllSam["blockCount"]=0
dfAllSam["blockSizes"]=""
dfAllSam["qStarts"]=""
dfAllSam["tStarts"]=""
dfAllSam[["PRE","POST","MPRER","MPRET","MPOSTR","MPOSTT","N","READ_LEN","DI","blockCount","blockSizes","qStarts"]]=pd.DataFrame(dfAllSam.apply(lambda row: se(row),axis=1))
dfAllSam=parseCIGAR(dfAllSam)
dfAllSam["tStarts"]=dfAllSam.apply(lambda row: tStarts(row),axis=1)
dfAllSam.drop(["FLAG","QUAL","paired","aligned2Mates","unmappedCurr","unmappedMate","reversedMate","firstRead","lastRead","secondaryAlignment","noPassFilter","PCRdup","suppAl","qStarts"],axis=1,inplace=True)
dfAllSam

,QNAME,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,MD,reversedCurr,PRE,POST,MPRER,MPRET,MPOSTR,MPOSTT,N,blockCount,blockSizes,tStarts,READ_LEN,DI,END,Template_start,Template_end,Reference_start,Reference_end
0,CHS.54819.2,CM000670.2,128564229,2,451M,*,0,0,TGTGCTCTGTGATGGCCAATGTAGTAGCATCGTGGAAAAATGGCCC...,451,16,0,0,0,0,451,451,0,1,451,128564229,451,0,451,0,451,128564229,128564679
1,CHS.54819.1,CM000670.2,128559028,2,549M663N309M521N57M3102N451M,*,0,0,tcatttaaacaaaaagtaataatttttattttcctaaaataaaaaa...,1366,16,0,0,549,549,817,817,4286,4,"549,309,57,451","128559028,128560240,128561733,128566076",1366,0,1366,0,1366,128559028,128564679
2,CHS.54822.1,CM000670.2,1371622,3,313M429N858M,*,0,0,ttaaatttgatGGTAATGAAACTGGATGTTGAAATTTGAGGTGAAT...,729A441,16,0,0,313,313,858,858,429,2,"313,858","1371622,1372364",1171,0,1171,0,1171,1371622,1373221
3,CHS.54821.1,CM000670.2,1296034,3,170M829N196M2696N2683M,*,0,0,TGTCTGCCAACAATCCTTTTATTTTGGCTTTACCTGCTGTTCGCCT...,3049,16,0,0,170,170,2879,2879,3525,3,"170,196,2683","1296034,1297033,1300754",3049,0,3049,0,3049,1296034,1302607
4,CHS.54824.2,CM000670.2,1428270,2,838M207N140M7453N205M,*,0,0,GCAATTTTatgatgtaaaatttaaaaagctagatAAAAAGGACAat...,1183,16,0,0,838,838,345,345,7660,3,"838,140,205","1428270,1429315,1437115",1183,0,1183,0,1183,1428270,1437112
5,CHS.54824.1,CM000670.2,1428270,40,838M207N140M130N395M,*,0,0,GCAATTTTatgatgtaaaatttaaaaagctagatAAAAAGGACAat...,1373,16,0,0,838,838,535,535,337,3,"838,140,395","1428270,1429315,1429792",1373,0,1373,0,1373,1428270,1429979
6,CHS.54828.1,CM000670.2,878741,3,858M2422N189M,*,0,0,TAAAGTGAGGCCACCTCTCCTGTTTGGCTTCTCTTTCACAAGGCCC...,1047,16,0,0,858,858,189,189,2422,2,"858,189","878741,882021",1047,0,1047,0,1047,878741,882209
7,CHS.54828.2,CM000670.2,878741,3,858M2793N553M,*,0,0,TAAAGTGAGGCCACCTCTCCTGTTTGGCTTCTCTTTCACAAGGCCC...,1411,16,0,0,858,858,553,553,2793,2,"858,553","878741,882392",1411,0,1411,0,1411,878741,882944
8,CHS.54827.1,CM000670.2,1565509,1,230M44S,*,0,0,tttatattcatttccttTTAGGATATGCATTTTTATACATCTGACT...,230,16,0,44,0,0,230,230,0,1,230,1565509,274,0,230,44,274,1565509,1565738
9,CHS.54830.1,CM000670.2,12392730,2,685M6893N58M,*,0,0,TTCCCAGTGGAAGTGTATATCTTAGAGTCAGTTTACCCTCCTGTCA...,241A19G279G201,16,0,0,685,685,58,58,6893,2,"685,58","12392730,12400308",743,0,743,0,743,12392730,12400365


In [6]:
dfAllSam["strand"]=np.where(dfAllSam["reversedCurr"]==16,"-","+")
dfAllSam

,QNAME,RNAME,POS,MAPQ,CIGAR,RNEXT,PNEXT,TLEN,SEQ,MD,reversedCurr,PRE,POST,MPRER,MPRET,MPOSTR,MPOSTT,N,blockCount,blockSizes,tStarts,READ_LEN,DI,END,Template_start,Template_end,Reference_start,Reference_end,strand
0,CHS.54819.2,CM000670.2,128564229,2,451M,*,0,0,TGTGCTCTGTGATGGCCAATGTAGTAGCATCGTGGAAAAATGGCCC...,451,16,0,0,0,0,451,451,0,1,451,128564229,451,0,451,0,451,128564229,128564679,-
1,CHS.54819.1,CM000670.2,128559028,2,549M663N309M521N57M3102N451M,*,0,0,tcatttaaacaaaaagtaataatttttattttcctaaaataaaaaa...,1366,16,0,0,549,549,817,817,4286,4,"549,309,57,451","128559028,128560240,128561733,128566076",1366,0,1366,0,1366,128559028,128564679,-
2,CHS.54822.1,CM000670.2,1371622,3,313M429N858M,*,0,0,ttaaatttgatGGTAATGAAACTGGATGTTGAAATTTGAGGTGAAT...,729A441,16,0,0,313,313,858,858,429,2,"313,858","1371622,1372364",1171,0,1171,0,1171,1371622,1373221,-
3,CHS.54821.1,CM000670.2,1296034,3,170M829N196M2696N2683M,*,0,0,TGTCTGCCAACAATCCTTTTATTTTGGCTTTACCTGCTGTTCGCCT...,3049,16,0,0,170,170,2879,2879,3525,3,"170,196,2683","1296034,1297033,1300754",3049,0,3049,0,3049,1296034,1302607,-
4,CHS.54824.2,CM000670.2,1428270,2,838M207N140M7453N205M,*,0,0,GCAATTTTatgatgtaaaatttaaaaagctagatAAAAAGGACAat...,1183,16,0,0,838,838,345,345,7660,3,"838,140,205","1428270,1429315,1437115",1183,0,1183,0,1183,1428270,1437112,-
5,CHS.54824.1,CM000670.2,1428270,40,838M207N140M130N395M,*,0,0,GCAATTTTatgatgtaaaatttaaaaagctagatAAAAAGGACAat...,1373,16,0,0,838,838,535,535,337,3,"838,140,395","1428270,1429315,1429792",1373,0,1373,0,1373,1428270,1429979,-
6,CHS.54828.1,CM000670.2,878741,3,858M2422N189M,*,0,0,TAAAGTGAGGCCACCTCTCCTGTTTGGCTTCTCTTTCACAAGGCCC...,1047,16,0,0,858,858,189,189,2422,2,"858,189","878741,882021",1047,0,1047,0,1047,878741,882209,-
7,CHS.54828.2,CM000670.2,878741,3,858M2793N553M,*,0,0,TAAAGTGAGGCCACCTCTCCTGTTTGGCTTCTCTTTCACAAGGCCC...,1411,16,0,0,858,858,553,553,2793,2,"858,553","878741,882392",1411,0,1411,0,1411,878741,882944,-
8,CHS.54827.1,CM000670.2,1565509,1,230M44S,*,0,0,tttatattcatttccttTTAGGATATGCATTTTTATACATCTGACT...,230,16,0,44,0,0,230,230,0,1,230,1565509,274,0,230,44,274,1565509,1565738,-
9,CHS.54830.1,CM000670.2,12392730,2,685M6893N58M,*,0,0,TTCCCAGTGGAAGTGTATATCTTAGAGTCAGTTTACCCTCCTGTCA...,241A19G279G201,16,0,0,685,685,58,58,6893,2,"685,58","12392730,12400308",743,0,743,0,743,12392730,12400365,-


In [7]:
# need to re-write the writeGFF function to make faster
includeCols=["matches",
                "misMatches",
                "repMatches",
                "nCount",
                "qNumInsert",
                "qBaseInsert",
                "tNumInsert",
                "tBaseInsert",
                "strand",
                "qName",
                "qSize",
                "qStart",
                "qEnd",
                "tName",
                "tSize",
                "tStart",
                "tEnd",
                "blockCount"]
splitCols=["blockSizes",
           "tStarts"]
# make unique id from index
dfAllSam["uid"]=dfAllSam.reset_index(drop=True).reset_index()["index"]

dfGFF=pd.DataFrame([])
for col in splitCols:
    tmp=pd.concat([pd.Series(row['uid'], row[col].split(','))              
                        for _, row in dfAllSam.iterrows()]).reset_index()
    tmp.columns=[col,"uid"]
    dfGFF=pd.concat([dfGFF,tmp],axis=1)
    
dfGFF=dfGFF[~(dfGFF["tStarts"]=="")].reset_index(drop=True) # get rid of empty lines
dfGFF.columns=["blockSizes","uid1","tStarts","uid"]
dfGFF.drop(["uid1"],axis=1,inplace=True)
dfAllSam.rename({"RNAME":'tName'},inplace=True,axis=1)
dfGFF=dfGFF.merge(dfAllSam[["tName","strand","uid"]],on="uid",how="left")
dfGFF["phase"]="."
dfGFF["score"]="."
dfGFF["ref"]="ref"
dfGFF["type"]="exon"
dfGFF["attribute"]=dfGFF["uid"]
dfGFF["start"]=dfGFF['tStarts']
dfGFF['end']=dfGFF['start'].astype(int)+dfGFF["blockSizes"].astype(int)
dfGFF=dfGFF[['tName',\
             'ref',\
             'type',\
             'start',\
             'end',\
             'score',\
             'strand',\
             'phase',\
             'attribute']]
# dfGFF.to_csv(outDir+"/gmap_aligned.gff",sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)
dfAllSam[["QNAME","uid"]].to_csv(outDir+"/gmap_map.csv",index=False)

In [8]:
# we can try to write out a gff of the blocks for the gff compare

# first need to add a transcript feature for each exon group
dft=dfGFF.groupby(by='attribute').agg({"start":"min","end":"max","ref":"min","tName":"min","score":"min","strand":"min","phase":"min"}).reset_index()
dft['type']='transcript'
dft['attribute']='ID='+dft['attribute'].astype(str)
dfGFF['attribute']='Parent='+dfGFF['attribute'].astype(str)
dfGFF_f=pd.concat([dft[['tName','ref','type','start','end','score','strand','phase','attribute']],dfGFF])
dfGFF_f.to_csv(outDir+'/dfGFF_gmap.gtf',sep="\t",index=False,header=False,quoting=csv.QUOTE_NONE)

# gffread -E dfGFF.gtf  -o- > dfGFF.gff